## Pràctica 9 - Homologia persistent i ML

Fins aquí hem après com d'un núvol de punts podem extreure una gràfica anomenada diagrama de persistència i que aquesta reflecteix d'alguna forma el tipus d'homotopia de la figura subjacent al núvol de punts.

Realment, no és tan interessant saber exactament quin és el tipus d'homotopia d'aquesta figura subjacent. De fet, potser no ens interessa en absolut. El que és important és que som capaços d'extreure un tipus d'informació de caràcter purament topològic, i per tant, relatiu d'alguna manera ala *forma* de les dades.

Aquesta informació topològica pot ser un tret diferencial del nostre núvol de punts que ens permeti distingir-lo d'altres.

En aquesta sessió veurem com podem combinar el que hem après d'homologia persistent amb machine learning.

Començarem amb un exemple ben clar. A la carpeta ```data``` disposeu d'un pickle que conté un Pandas DataFrame amb una colecció etiquetada de núvols de punts sobre $\mathbb{R}^3$.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_pickle("data/3dshapes.pkl")
df

,points,label
0,"[[-15.926158536980935, -44.52969502310438, -40...",A
1,"[[-41.21983471453681, -11.223441426171235, -22...",B
2,"[[8.984239528308677, -92.7367011188133, -122.8...",C
3,"[[-83.71552235245441, -76.52368688771078, -79....",B
4,"[[69.26248812416945, 86.36564268523749, -21.83...",C
5,"[[-21.920179302268807, -62.43303153577968, 12....",A
6,"[[-81.48289484783425, -68.73266737425021, -95....",B
7,"[[-112.51341042642444, 41.75960310484143, -200...",C
8,"[[-138.34742017057124, -50.416950027295826, -7...",A
9,"[[-117.89004538750717, -46.21937596797612, -35...",B


Si us falla la lectura del pickle (que és possible perquè la *picklelització* depèn de l'environment on s'ha creat), disposeu d'un JSON que conté les mateixes dades. Amb JSON, però, les ```np.arrays``` es transformen a llistes, i per això, per a poder seguir el que fem en aquesta notebook, heu de convertir la columna *point* a ```np.array```.

In [ ]:
df = pd.read_json("data/3dshapes.json")
df["points"] = df["points"].apply(np.array)
df

,points,label
0,"[[-15.926158537, -44.5296950231, -40.535997221...",A
1,"[[-41.2198347145, -11.2234414262, -22.41602786...",B
2,"[[8.9842395283, -92.7367011188, -122.854768593...",C
3,"[[-83.7155223525, -76.5236868877, -79.55095900...",B
4,"[[69.2624881242, 86.3656426852, -21.8354491306...",C
5,"[[-21.9201793023, -62.4330315358, 12.622511887...",A
6,"[[-81.4828948478, -68.73266737430001, -95.4064...",B
7,"[[-112.5134104264, 41.7596031048, -200.2539254...",C
8,"[[-138.3474201706, -50.4169500273, -77.0289618...",A
9,"[[-117.8900453875, -46.219375968, -35.13303369...",B


**Dibuixeu un scatterplot 3d d'algunes d'aquestes figures per endevinar (a ull) a qué corresponen les etiquetes A, B i C**

A ull és molt fàcil. Ara bé, com faríem si volguéssim automatitzar aquesta classificació? Amb TDA, evidentment.

**Dibuixeu per a cadascun dels individus el seu diagrama de persistència**

Efectivament, podem veure com aquests diagrames de persistència son molt característic de cadascuna de les classes, per tant son un gran classificador.

### Feature extraction

El problema és que un diagrama de persistència segueix sense ser una dada numèrica que poguem utilizar de forma objectiva. Segueix havent-hi una component humana quan diem que aquests diagrames son caracterítics de cada classe; ho estem dient a ull.

Aquí s'obre un nou punt d'interès i és: com podem extreure features numèriques dels diagrames de persistència?

L'aproximació més simple ens la dóna el que s'anomena *entropia de persistència* que mesura com de caòtica és la distribució de punts en cada dimensió.

**Calculeu la entropia de persistència dels núvols de punts i poseu-los com a noves columnes del dataframe. Valoreu els resultats obtinguts.**

Com podeu observar, aquestes variables que ens dóna la entropia de persistència recullen de bona manera el que teníem als diagrames de persistècia, i es converteixen així en uns excelents discriminadors.

**Monteu un model de ML amb scikit-learn (el que volgueu) que classifiqui els núvols de punts**

El rendiment molt probablement us doni impecable. També és cert que son unes dades sintèntiques i molt "acadèmiques", i. e., molt preparades per a que funcionin bé com a exemple.

Igualment val la pena destacar vàries coses:
- Els núvols de punts estaven etiquetats en funció de la forma que tenien, i la homologia persistent ha sabut captar això.
- No existeixen altres opcions per extreure informació de la forma. *Si no sapiguéssiu TDA, sabríeu muntar un model que classifiqui això?*
- La informació extreta és purament topològica, no depèn de deformacions, rotacions o traslacions. Si us fixeu, els núvols de punts estaven rotats aleatòriament en l'espai, tenien diferents escales, i estaven centrats en llocs diferents.

Tot i així, és important deixar clar que un anàlisi com aquest podria permetre discriminar objectes amb tipus d'homotopia igual, ja que recordem que la homologia persistent no són els grups d'homologia de cap figura en particular, sino d'un continu. La manera en la que avança aquest continu és una cosa que depèn de la forma de la nostra figura més enllà del seu tipus homotòpic.

**Carregueu ara el conjunt de dades ```data/more3dshapes.pkl``` i repetiu el procediment fet avui a classe**